In [ ]:
#pip install cloudinary

## Solution 1: Upload jpg file to cloudinary 

Current Approach

In [10]:
import os
import cloudinary.uploader

import numpy as np
import json

In [ ]:
# setup cloudinary

In [ ]:
np.random.seed(10)
num_imgs_upload = 1000
random_numbers = np.random.randint(0, 33502, 10)

In [ ]:
def uploadImage(local_path, id):
    cloudinary.uploader.upload(local_path,
                               folder='DSC106 MET Images',
                               public_id=id)

def get_image_data(root_folder):
    image_paths = []
    path_base = 'https://res.cloudinary.com/duyoevfl6/image/upload/DSC106%20MET%20Images/'
    i = 0
    for subdir, _, files in os.walk(root_folder):
        # only takes the first image file in each subdire folder
        img_file = files[0]
        if img_file.lower().endswith(('.jpg')):  # checking the extension of the files

            image_path = os.path.join(subdir, img_file)
            id = subdir[subdir.rfind('/')+1:]

            uploadImage(image_path, id)
            img_path = {'id': id, 'path': path_base + id + '.jpg'}

            image_paths.append(img_path)

            # only upload 20 images
            i+=1
            if i > 19:
                return image_paths
    return 1

image_directory = 'data/MET'
img_path_data = get_image_data(image_directory)

In [ ]:
# with open('small_img_data.json', 'w') as json_file:
#     json.dump(img_path_data, json_file)

## Solution 2: Convert jpg file to svg file

Does not work as intended -> file size does not decrease

In [23]:
import cv2
import numpy as np
import svgwrite

# def convert_jpg_to_svg(jpg_path, svg_path):
#     # Read the image
#     img = cv2.imread(jpg_path, cv2.IMREAD_GRAYSCALE)
    
#     # Threshold the image to create a binary image
#     _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)

#     # Find contours
#     contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#     # Create an SVG drawing
#     dwg = svgwrite.Drawing(svg_path, profile='tiny')

#     # Add contours to the SVG drawing
#     for contour in contours:
#         path_data = "M " + " L ".join(f"{pt[0][0]},{pt[0][1]}" for pt in contour) + " Z"
#         dwg.add(dwg.path(d=path_data, fill='none', stroke='black'))

#     # Save the SVG file
#     dwg.save()

def convert_color_jpg_to_svg(jpg_path, svg_path):
    # Read the image in color
    img = cv2.imread(jpg_path)

    # Convert the image to RGB (OpenCV loads images in BGR by default)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply Gaussian Blur to reduce noise and smooth the image
    img_blur = cv2.GaussianBlur(img, (5, 5), 0)

    # Convert the image to a binary image
    gray = cv2.cvtColor(img_blur, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an SVG drawing
    dwg = svgwrite.Drawing(svg_path, profile='tiny')

    # Add contours to the SVG drawing
    for contour in contours:
        # Create a mask for each contour
        mask = np.zeros_like(img)
        cv2.drawContours(mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

        # Extract the color within the contour from the original image
        color = cv2.mean(img, mask=mask[:,:,0])[:3]  # Get the RGB color

        # Convert the contour to a path string
        path_data = "M " + " L ".join(f"{pt[0][0]},{pt[0][1]}" for pt in contour) + " Z"

        # Add the path to the SVG drawing with the extracted color
        dwg.add(dwg.path(d=path_data, fill=svgwrite.rgb(color[0], color[1], color[2]), stroke='none'))

    # Save the SVG file
    dwg.save()

In [24]:
def get_image_data(root_folder):
    image_paths = []

    i = 0
    for subdir, _, files in os.walk(root_folder):
        # only takes the first image file in each subdire folder
        img_file = files[0]
        if img_file.lower().endswith(('.jpg')):  # checking the extension of the files

            image_path = os.path.join(subdir, img_file)
            
            convert_jpg_to_svg(image_path, 'test.svg')

            # only upload 20 images
            i+=1
            if i > 0:
                return image_paths
    return 1

image_directory = 'data/MET'
# img_path_data = get_image_data(image_directory)

data/MET/390655/1.jpg


In [ ]:
# from PIL import Image
# import svgwrite

# def convert_jpg_to_svg(jpg_path, svg_path):
#     # Open the image and convert it to grayscale
#     image = Image.open(jpg_path).convert('L')
    
#     # Threshold the image to make it binary (black and white)
#     threshold = 128
#     image = image.point(lambda p: p > threshold and 255)
#     image = image.convert('1')
    
#     # Create a bitmap from the image
#     bitmap = potrace.Bitmap(image)
    
#     # Trace the bitmap to a path
#     path = bitmap.trace()
    
#     # Create an SVG drawing
#     dwg = svgwrite.Drawing(svg_path, profile='tiny')
    
#     # Add paths to the SVG drawing
#     for curve in path:
#         start_point = curve.start_point
#         d = f'M {start_point.x} {start_point.y} '
#         for segment in curve:
#             if segment.is_corner:
#                 d += f'L {segment.c.x} {segment.c.y} '
#             else:
#                 d += f'C {segment.c1.x} {segment.c1.y} {segment.c2.x} {segment.c2.y} {segment.end_point.x} {segment.end_point.y} '
#         d += 'Z'
#         dwg.add(dwg.path(d, fill='black'))
    
#     # Save the SVG file
#     dwg.save()

# # Convert the JPG to SVG
# convert_jpg_to_svg('input.jpg', 'output.svg')